In [ ]:
import requests
import pandas as pd
import time
import json
# Define your API key and endpoint
ENDPOINT = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#csv to pandas
df = pd.read_csv('orenda_Kyiv_lun_rieltor.csv')

#iterate over dataframe
for index, row in df.iterrows():
    latitude = row.latitude
    longitude = row.longitude

    params = {
        'key': API_KEY,
        'location': f'{latitude},{longitude}',
        'radius': 10000,
        'type': 'subway_station'
    }

    # Send the nearby search request
    response = requests.get(ENDPOINT, params=params)
    results = response.json()
    json_string = json.dumps(results, indent=4)

    print(json_string)

    # if results not empty
    if len(results['results']) > 0:
        # Access the details of the closest subway station
        closest_station = results['results'][0]
        name = closest_station['name']
        l = closest_station['geometry']['location']
        location = f'{l["lat"]}, {l["lng"]}'

        # Display the details of the closest subway station
        print('Closest Subway Station:')
        print('Name:', name)
        print('Location:', location)

        df.at[index, "Closest_subway"] = name
        df.at[index, "Closest_subway_location"] = location

        df.to_csv('rieltor_subways.csv', index=False)

In [55]:
import googlemaps

gmaps = googlemaps.Client(key=API_KEY)
df = pd.read_csv('rieltor_subways.csv')

for index, row in df.iterrows():
    lat = row.latitude
    lng = row.longitude

    if pd.isna(row.Closest_subway_location):
        continue

    loc = row.Closest_subway_location.split(',')
    loc = [float(i) for i in loc]

    a = (lat, lng)  # Latitude and longitude of point A
    b = (loc[0], loc[1])  # Latitude and longitude of point B

    result = gmaps.distance_matrix(a, b, mode='walking', units='metric')
    r = json.dumps(result, indent=4)

    df.at[index, "Distance Text"] = result['rows'][0]['elements'][0]['distance']['text']
    df.at[index, "Distance Value"] = result['rows'][0]['elements'][0]['distance']['value']
    df.at[index, "Duration Text"] = result['rows'][0]['elements'][0]['duration']['text']
    df.at[index, "Duration Value"] = result['rows'][0]['elements'][0]['duration']['value']

    df.to_csv('rieltor_subways.csv', index=False)
